In [ ]:
from mongo_episode import Episode
from mongo_auteur import Auteur, AuthorChecker
import pandas as pd


def ajoute_auteurs(episode: Episode, verbose=False):
    affiche_episode = f"""
    Date: {episode.date.strftime("%d %b %Y")}
    Titre: {episode.titre}
    Description: {episode.description}

    """
    print(affiche_episode)

    auteurs = episode.get_all_auteurs()
    print(f"La liste des auteurs : {auteurs}\n")

    ac = AuthorChecker(episode)
    auteur_traitement_df = pd.DataFrame(
        columns=["auteur_corrige", "detection", "existait_en_base", "anomalie"]
    )
    analyse_dict = {}

    for auteur in auteurs:
        auteur_corrige_dict = ac.check_author(
            auteur, return_details=True, verbose=verbose
        )
        if verbose:
            print(auteur_corrige_dict)
        # check_author retourne None dans author_corrected si le process via rss:metadata, db, llm, web search n'a rien renvoye
        # c'est donc une anomalie
        anomalie = True if auteur_corrige_dict["author_corrected"] is None else False
        # dans auteur_corrige c'est le nom de l'auteur si il est different de l'original sinon None
        auteur_corrige = (
            auteur_corrige_dict["author_corrected"]
            if auteur_corrige_dict["author_corrected"]
            != auteur_corrige_dict["author_original"]
            else None
        )
        nom_auteur = auteur_corrige_dict["author_corrected"] if not None else auteur
        # a quel endroit a ete detecte l'auteur (rss:metadata, db, llm, web search)
        detection = auteur_corrige_dict["source"]
        existait_en_base = False if anomalie else Auteur(nom_auteur).exists()
        auteur_traitement_df.loc[auteur] = [
            auteur_corrige,
            detection,
            existait_en_base,
            anomalie,
        ]
        if not anomalie:
            aut = Auteur(nom_auteur)
            aut.keep()
        else:
            analyse_dict[auteur] = {
                "analyse": auteur_corrige_dict["analyse"],
                "score": auteur_corrige_dict["score"],
            }

    print(episode.date)
    display(auteur_traitement_df)
    if len(analyse_dict) > 0:
        print("\nAnalyse des anomalies\n")
    for auteur, details in analyse_dict.items():
        print(f"{auteur} -> {details['analyse']} (score: {details['score']})")

# scripts/store_all_auteurs_from_episode.py

In [ ]:
import datetime

date = datetime.datetime(2021, 1, 17)

episode = Episode.from_date(date=date)
assert episode is not None, "no episode for date {}".format(date)

# ajoute_auteurs(episode, verbose=True)

# scripts/store_all_auteurs_from_all_episodes.py

In [ ]:
verbose = True

In [ ]:
import os
from git import Repo


def get_git_root(path: str) -> str:
    """Retrieves the root directory of the Git repository.

    Args:
        path (str): The current working directory.

    Returns:
        str: The root directory of the Git repository.
    """
    git_repo = Repo(path, search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")


project_root: str = get_git_root(os.getcwd())
cache_filename: str = os.path.join(
    project_root, "scripts", "store_all_auteurs_from_all_episodes.txt"
)
cache_filename

'/home/guillaume/git/lmelp/scripts/store_all_auteurs_from_all_episodes.txt'

In [ ]:
import os
import datetime
from mongo_episode import Episodes

# lis le contenu du fichier cache_filename s'il existe
if os.path.exists(cache_filename):
    with open(cache_filename, "r") as f:
        date_cache = datetime.datetime.strptime(f.read(), "%d/%m/%Y")
        print(f"Reprise du traitement à partir de la date {date_cache}")
else:
    date_cache = datetime.datetime.now()

date = datetime.datetime.now()

# on prend la date la plus ancienne entre date et date_cache
date = date if date < date_cache else date_cache
episodes = Episodes().get_entries({"date": {"$lt": date}})
for episode in episodes:
    if episode.date < date:
        ajoute_auteurs(episode, verbose=verbose)
        # on sauvegarde la date de traitement
        # le cache ne contient que cette date, rien d'autre
        with open(cache_filename, "w") as f:
            f.write(episode.date.strftime("%d/%m/%Y"))

Reprise du traitement à partir de la date 2020-11-29 00:00:00

    Date: 15 Nov 2020
    Titre: Ces auteurs à retrouver en librairie : Michel Houellebecq, Patti Smith, Simone de Beauvoir…
    Description: Que
 pensent nos critiques de "L’année du singe" de Patti Smith, 'Le banquet
 annuel de la confrérie des fossoyeurs' de Mathias Enard, "Interventions
 2020" de Michel Houellebecq, "La famille Martin" de David Foenkinos, 
"Les inséparables" de Simone de Beauvoir ?

    
La liste des auteurs : ['Patti Smith', 'Mathias Enard', 'Michel Houellebecq', 'David Foenkinos', 'Simone de Beauvoir', 'Hervé Le Tellier', 'Ken Follett', 'Léna Situations', 'Jean-Claude Raspien-Jas', 'Frédéric Beigbeder', 'Arnaud Viviant', 'Olivia de Lamberterie', 'Jean-Louis Zambon', 'Michel Moreau', 'Nelly Kaprièlian', 'Jérôme Duvivier', 'Stéphanie Sorin', 'Pauline Salvi', 'Pascal Gervais', 'Loïc Prigent', 'Marie Kondo', 'Tristan Garcia', 'Pierre Loti', 'Emmanuel Carrère', 'Philippe Muray', 'Victor Hugo', 'Jean Genet'

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': True, 'severity': 'medium'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}